<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/13-PySpark/spark_streaming_triggering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession


if __name__=="__main__":
    spark = SparkSession.builder.appName("spark_streaming").config("spark.sql.shuffle.partitions",3)\
        .config("spark.streaming.stopGracefullyOnShutdown",True).getOrCreate()

    df_ = spark.readStream.format("socket").option("host","localhost").option("port","9093").load()

    df1 = df_.selectExpr("explode(split(value,' ')) as word")
    df2 = df1.groupBy("word").count()

    result = df2.writeStream.format("console").outputMode("complete").option("checkpointLocation","checkppoint-location1")\
        .trigger(processingTime="30 seconds").start()
    result.awaitTermination()

In [ ]:
from pyspark.sql import SparkSession


if __name__=="__main__":
    spark = SparkSession.builder.appName("spark_streaming").config("spark.sql.shuffle.partitions",3)\
        .getOrCreate()

    df_ = spark.readStream.format("socket").option("host","localhost").option("port","9092").load()

    #split will always give array
    #Explode on the array to get each words in different row.

    df1 = df_.selectExpr("explode(split(value,' ')) as word")

    df2 = df1.groupBy("word").count()

    ##write to console
    result = df2.writeStream.format("console").outputMode("complete")\
        .option("checkpointLocation","checkppoint-location1")\
        .start()


    result.awaitTermination()

In [ ]:
from pyspark.sql import SparkSession


if __name__=="__main__":
    spark =SparkSession.builder.appName("spark_streaming").getOrCreate()


    ##read the stream

    df_ = spark.readStream.format("socket").option("host","localhost").option("port","9092").load()

    #Print the schema
    df_.printSchema()

    ##write to sink
    result = df_.writeStream.format("console").outputMode("append").start()


    result.awaitTermination()